<a href="https://colab.research.google.com/github/GokuldasCMP/Cold-Email-Generator/blob/master/Colab%20Notebooks/email_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
!pip install langchain


In [34]:
%pip install -qU langchain-groq

In [35]:
from langchain_groq import ChatGroq


In [36]:

llm = ChatGroq(
    temperature=0,
    groq_api_key='Your API Key',
    model_name="llama-3.1-70b-versatile"
)
response = llm.invoke("The first person to land on moon was ...")
print(response.content)

The first person to land on the moon was Neil Armstrong. He stepped onto the lunar surface on July 20, 1969, as part of the Apollo 11 mission.


In [37]:
%pip install -qU langchain_community beautifulsoup4

In [46]:
# WebBaseLoader class in langchain will accept the url of the job posting and extract data from that.
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-39383?from=job%20search%20funnel")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer II - Workday Integration, ITC

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer II - Workday Integration, ITCKarnataka, IndiaBecome

In [47]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):
        """
)

chain_extract = prompt_extract | llm  #use a pipe operation. This will form the chain
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [48]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res

{'role': 'Software Engineer II - Workday Integration, ITC',
 'experience': '5+ years of experience building integrations and software development',
 'skills': ['Workday Studio, EIB, Core Connectors and payroll connectors (PECI)',
  'XML, XSLT',
  'Web Services (WSDL, SOAP), EDI and Flat file integrations',
  'SaaS applications, SaaS deployment models, and SaaS DevOps',
  'Object Oriented concepts',
  'Agile Framework'],
 'description': 'We’re looking for a Workday Integration Specialist to support HR Services and Operations. This role will sit within our HR Technology Engineering - the team who supports Workday Integration solutions for Core HR Shared Services, Payroll, Total Rewards Operations, Talent Operations, Talent Acquisition, and Learning & Development.'}

In [49]:
type(json_res)

dict

In [50]:
import pandas as pd

df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Cold-Email-Generator/my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [51]:
!pip install chromadb

In [52]:
import uuid
import chromadb

client = chromadb.PersistentClient('/content/drive/MyDrive/Colab Notebooks/Cold-Email-Generator/vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [53]:
job = json_res
job['skills']

['Workday Studio, EIB, Core Connectors and payroll connectors (PECI)',
 'XML, XSLT',
 'Web Services (WSDL, SOAP), EDI and Flat file integrations',
 'SaaS applications, SaaS deployment models, and SaaS DevOps',
 'Object Oriented concepts',
 'Agile Framework']

In [54]:
job

{'role': 'Software Engineer II - Workday Integration, ITC',
 'experience': '5+ years of experience building integrations and software development',
 'skills': ['Workday Studio, EIB, Core Connectors and payroll connectors (PECI)',
  'XML, XSLT',
  'Web Services (WSDL, SOAP), EDI and Flat file integrations',
  'SaaS applications, SaaS deployment models, and SaaS DevOps',
  'Object Oriented concepts',
  'Agile Framework'],
 'description': 'We’re looking for a Workday Integration Specialist to support HR Services and Operations. This role will sit within our HR Technology Engineering - the team who supports Workday Integration solutions for Core HR Shared Services, Payroll, Total Rewards Operations, Talent Operations, Talent Acquisition, and Learning & Development.'}

In [55]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

[[{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/magento-portfolio'},
  {'links': 'https://example.com/wordpress-portfolio'}],
 [{'links': 'https://example.com/wordpress-portfolio'},
  {'links': 'https://example.com/magento-portfolio'}],
 [{'links': 'https://example.com/devops-portfolio'},
  {'links': 'https://example.com/angular-portfolio'}],
 [{'links': 'https://example.com/java-portfolio'},
  {'links': 'https://example.com/android-portfolio'}],
 [{'links': 'https://example.com/ios-ar-portfolio'},
  {'links': 'https://example.com/kotlin-backend-portfolio'}]]

In [56]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}

        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools.
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability,
        process optimization, cost reduction, and heightened overall efficiency.
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ.
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):

        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Workday Integration Solutions for Seamless HR Operations

Dear Hiring Manager,

I came across the job posting for a Software Engineer II - Workday Integration, ITC, and I'm excited to introduce AtliQ, an AI & Software Consulting company that can help you achieve your Workday integration goals.

With over 5+ years of experience in building integrations and software development, our team at AtliQ has the expertise to support your HR Services and Operations. We have a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency.

Our team is well-versed in Workday Studio, EIB, Core Connectors, and payroll connectors (PECI), as well as XML, XSLT, Web Services (WSDL, SOAP), EDI, and Flat file integrations. We also have expertise in SaaS applications, SaaS deployment models, and SaaS DevOps, ensuring seamless integration with your existing systems.

AtliQ's portfolio showcases our capabil